# Directing notebook cell execution output to a custom VS Code notebook renderer

Switch the cell output rendering by clicking on the [...] button next to the cell output and then _Change Presentation_. When you select the `x-application/github-issues`, the custom renderer will be invoked.

If you are getting _Renderer not available_ message, you are not running/debugging this sample extension.

## Using `IPython.display.display` function and mimetypes

In [1]:
from IPython import display
import json

issues = [{
        "url": "https://github.com/project/repo/issue/123",
        "title": "Bug A",
        "body": "Bug A description..."
    },
    {
        "url": "https://github.com/project/repo/issue/456",
        "title": "Bug B",
        "body": "Bug B description..."
    }]

display.display({
    "x-application/github-issues": json.dumps(issues),
    "application/json": json.dumps(issues, indent=2),
    "text/html": "<h1>[html] " + ", ".join([i["title"] for i in issues]) + "</h1>",
    "text/plain": "[plain] " + ", ".join([i["url"] for i in issues]),
}, raw=True)

[plain] https://github.com/project/repo/issue/123, https://github.com/project/repo/issue/456

# Using the `IPython.display.display` function with an object

In [ ]:
from typing import List
from dataclasses import dataclass, is_dataclass, asdict

@dataclass
class GitHubIssue:
    url: str
    title: str
    body: str

issues = [GitHubIssue("https://github.com/project/repo/issue/123", "Bug A", "Bug A description..."),
    GitHubIssue("https://github.com/project/repo/issue/456", "Bug B", "Bug B description...")]

class DataClassJSONEncoder(json.JSONEncoder):
    """ Overrides the json serialization of dataclasses """
    def default(s, o):
        if is_dataclass(o):
            return asdict(o)
        return super().default(o)

class GitHubIssues(object):
    def __init__(self, issues: List[GitHubIssue]):
        self.issues = issues

    def json_dump(self) -> str:
        return json.dumps(self.issues, cls=DataClassJSONEncoder)

    def _repr_mimebundle_(self, include, exclude):
        return {
            "text/plain": "[plain]\n" + "\n".join([i.url for i in self.issues]),
            "text/html": "<h1>[html]</h1>" + "".join(['<li>' + i.title + '</li>' for i in self.issues]),
            "x-application/github-issues": self.json_dump(),
        }

display.display(GitHubIssues(issues))